<a href="https://colab.research.google.com/github/27luiz/dadosSUS/blob/main/Dadosus_AIH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import auth
auth.authenticate_user()


In [20]:
from ftplib import FTP
from google.cloud import storage

def download_ftp_file(ftp_url, local_filename):
    ftp_server = ftp_url.split('/')[2]
    ftp_path = "/".join(ftp_url.split('/')[3:])

    with FTP(ftp_server) as ftp:
        ftp.login()  # Você pode precisar fornecer credenciais de login aqui
        with open(local_filename, 'wb') as f:
            ftp.retrbinary(f'RETR {ftp_path}', f.write)

def upload_to_gcs(local_filename, gcs_bucket_name, gcs_blob_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(gcs_bucket_name)
    blob = bucket.blob(gcs_blob_name)
    blob.upload_from_filename(local_filename)

# URL base do arquivo FTP
ftp_base_url = "ftp://ftp.datasus.gov.br/dissemin/publicos/SIHSUS/200801_/Dados/RD{uf}{ano}{mes}.dbc"

# Localização no Google Cloud Storage para onde o arquivo será carregado
gcs_bucket_name = "tabsus"

# Listas de Ufs, Anos e Meses
ufs = ['BA']
anos = ['23']
meses = ['01', '02', '03']

# Iterar sobre todas as combinações de Ufs, Anos e Meses
for uf in ufs:
    for ano in anos:
        for mes in meses:
            # Construir a URL do FTP substituindo as partes fixas pelas variáveis
            ftp_url = ftp_base_url.format(uf=uf, ano=ano, mes=mes)

            # Construir o nome do arquivo local substituindo as partes fixas pelas variáveis
            local_filename = f"/tmp/RD{uf}{ano}{mes}.dbc"

            # Construir o nome do arquivo no Google Cloud Storage substituindo as partes fixas pelas variáveis
            gcs_blob_name = f"bruto/RD{uf}{ano}{mes}.dbc"

            # Baixe o arquivo FTP
            download_ftp_file(ftp_url, local_filename)

            # Carregue o arquivo baixado para o Google Cloud Storage
            upload_to_gcs(local_filename, gcs_bucket_name, gcs_blob_name)

            print(f"Arquivo enviado com sucesso para o Google Cloud Storage: {gcs_blob_name}")

print("Todos os arquivos foram enviados com sucesso.")

Arquivo enviado com sucesso para o Google Cloud Storage: bruto/RDBA2301.dbc
Arquivo enviado com sucesso para o Google Cloud Storage: bruto/RDBA2302.dbc
Arquivo enviado com sucesso para o Google Cloud Storage: bruto/RDBA2303.dbc
Todos os arquivos foram enviados com sucesso.


In [27]:
from ftplib import FTP
from google.cloud import storage
import threading

def download_and_upload(uf, ano, mes, gcs_bucket_name):
    ftp_url = ftp_base_url.format(uf=uf, ano=ano, mes=mes)
    local_filename = f"RD{uf}{ano}{mes}.dbc"
    gcs_blob_name = f"bruto/RD{uf}{ano}{mes}.dbc"

    try:
        # Download do arquivo FTP
        download_ftp_file(ftp_url, local_filename)

        # Upload do arquivo para o Google Cloud Storage
        upload_to_gcs(local_filename, gcs_bucket_name, gcs_blob_name)
    except Exception as e:
        print(f"Erro ao processar UF={uf}, Ano={ano}, Mês={mes}: {e}")

def download_ftp_file(ftp_url, local_filename):
    ftp_server = ftp_url.split('/')[2]
    ftp_path = "/".join(ftp_url.split('/')[3:])

    with FTP(ftp_server) as ftp:
        ftp.login()
        with open(local_filename, 'wb') as f:
            ftp.retrbinary(f'RETR {ftp_path}', f.write)

def upload_to_gcs(local_filename, gcs_bucket_name, gcs_blob_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(gcs_bucket_name)
    blob = bucket.blob(gcs_blob_name)
    blob.upload_from_filename(local_filename)
    print(f"Arquivo enviado com sucesso para o Google Cloud Storage: {gcs_blob_name}")

# URL base do arquivo FTP
ftp_base_url = "ftp://ftp.datasus.gov.br/dissemin/publicos/SIHSUS/200801_/Dados/RD{uf}{ano}{mes}.dbc"

# Localização no Google Cloud Storage para onde o arquivo será carregado
gcs_bucket_name = "tabsus"

# Listas de Ufs, Anos e Meses
ufs = ['Pi']
anos = ['23']
meses = ['01', '02', '03']

# Lista para armazenar as threads
threads = []

# Função para criar e iniciar threads
def create_and_start_thread(uf, ano, mes):
    thread = threading.Thread(target=download_and_upload, args=(uf, ano, mes, gcs_bucket_name))
    thread.start()
    threads.append(thread)

# Criar e iniciar threads para cada combinação de UF, ano e mês
for uf in ufs:
    for ano in anos:
        for mes in meses:
            create_and_start_thread(uf, ano, mes)

# Aguardar todas as threads terminarem
for thread in threads:
    thread.join()

print("Todos os arquivos foram enviados com sucesso.")

Arquivo enviado com sucesso para o Google Cloud Storage: bruto/RDPi2301.dbc
Arquivo enviado com sucesso para o Google Cloud Storage: bruto/RDPi2303.dbc
Arquivo enviado com sucesso para o Google Cloud Storage: bruto/RDPi2302.dbc
Todos os arquivos foram enviados com sucesso.


In [28]:
from ftplib import FTP
from google.cloud import storage
import threading
from datetime import datetime, timedelta

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days) + 1):
        yield start_date + timedelta(n)

def download_and_upload(date, gcs_bucket_name):
    ano = date.strftime("%y")
    mes = date.strftime("%m")
    uf = 'RJ'  # Aqui você pode definir a UF desejada

    ftp_url = ftp_base_url.format(uf=uf, ano=ano, mes=mes)
    local_filename = f"RD{uf}{ano}{mes}.dbc"
    gcs_blob_name = f"bruto/RD{uf}{ano}{mes}.dbc"

    try:
        # Download do arquivo FTP
        download_ftp_file(ftp_url, local_filename)

        # Upload do arquivo para o Google Cloud Storage
        upload_to_gcs(local_filename, gcs_bucket_name, gcs_blob_name)
    except Exception as e:
        print(f"Erro ao processar UF={uf}, Ano={ano}, Mês={mes}: {e}")

def download_ftp_file(ftp_url, local_filename):
    ftp_server = ftp_url.split('/')[2]
    ftp_path = "/".join(ftp_url.split('/')[3:])

    with FTP(ftp_server) as ftp:
        ftp.login()
        with open(local_filename, 'wb') as f:
            ftp.retrbinary(f'RETR {ftp_path}', f.write)

def upload_to_gcs(local_filename, gcs_bucket_name, gcs_blob_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(gcs_bucket_name)
    blob = bucket.blob(gcs_blob_name)
    blob.upload_from_filename(local_filename)
    print(f"Arquivo enviado com sucesso para o Google Cloud Storage: {gcs_blob_name}")

# URL base do arquivo FTP
ftp_base_url = "ftp://ftp.datasus.gov.br/dissemin/publicos/SIHSUS/200801_/Dados/RD{uf}{ano}{mes}.dbc"

# Localização no Google Cloud Storage para onde o arquivo será carregado
gcs_bucket_name = "tabsus"

# Intervalo de datas
start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 1, 1)

# Lista para armazenar as threads
threads = []

# Criar e iniciar threads para cada data no intervalo
for single_date in daterange(start_date, end_date):
    thread = threading.Thread(target=download_and_upload, args=(single_date, gcs_bucket_name))
    thread.start()
    threads.append(thread)

# Aguardar todas as threads terminarem
for thread in threads:
    thread.join()

print("Todos os arquivos foram enviados com sucesso.")


Arquivo enviado com sucesso para o Google Cloud Storage: bruto/RDRJ2401.dbc
Todos os arquivos foram enviados com sucesso.


In [30]:
pip install dbfread

In [31]:
import os
from google.cloud import storage
from dbfread import DBF

def convert_and_upload(source_blob_name, destination_blob_name):
    # Download do arquivo do Google Cloud Storage
    source_blob = storage_client.bucket(gcs_bucket_name).blob(source_blob_name)
    source_filename = "/tmp/input.dbf"  # Caminho local para salvar o arquivo temporariamente
    source_blob.download_to_filename(source_filename)

    # Converter o arquivo DBF para CSV
    output_filename = "/tmp/output.csv"  # Caminho local para salvar o arquivo convertido
    with open(output_filename, 'w') as f:
        for record in DBF(source_filename):
            f.write(','.join(map(str, record.values())))
            f.write('\n')

    # Upload do arquivo convertido para o Google Cloud Storage
    destination_blob = storage_client.bucket(gcs_bucket_name).blob(destination_blob_name)
    destination_blob.upload_from_filename(output_filename)

    print(f"Arquivo convertido e enviado com sucesso para: gs://{gcs_bucket_name}/{destination_blob_name}")

# Inicializar o cliente do Google Cloud Storage
storage_client = storage.Client()

# Nome do bucket
gcs_bucket_name = "tabsus"

# Nome do arquivo de entrada e saída
source_blob_name = "bruto/RDBA2301.dbc"
destination_blob_name = "tratado/RDBA2301.csv"

# Executar a conversão e upload
convert_and_upload(source_blob_name, destination_blob_name)

Arquivo convertido e enviado com sucesso para: gs://tabsus/tratado/RDBA2301.csv


In [36]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=5dc590bdb9b97d3bf630b8d199d87010a280bef52cb86323c459c5ac4a4cff0f
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [43]:
pip install dbc_reader

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for dbc_reader: filename=dbc_reader-0.1.2-py3-none-any.whl size=18497 sha256=26a96779ceb23d4941d5137b208c66a3a147a87b62fc2a79b0937e2d8a2429e8
  Stored in directory: /root/.cache/pip/wheels/71/de/52/658cf7a77a062a6c35ce3eac5cd4f43502ddf7f60b5a5701b1
  Created wheel for dbf-reader: filename=dbf_reader-0.2.2-py3-none-any.whl size=7627 sha256=5bd31a5c413e2a366b790deb190a256fe1af54b66870064f1f7d522b2fc0210c
  Stored in directory: /root/.cache/pip/wheels/ae/84/c4/e885b4d466946c4fd2473c1390a429ee4eec4cac53223413b8
Successfully built dbc_reader dbf-reader


In [64]:
from google.cloud import storage
from dbc_reader import DbcReader

# Inicializar o cliente do Google Cloud Storage
storage_client = storage.Client()

# Caminho do arquivo DBC no Google Cloud Storage
gcs_bucket_name = "tabsus"
blob_name = "bruto/RDBA2301.dbc"
local_file_path = "/tmp/RDBA2301.dbc"

# Baixar o arquivo DBC localmente
bucket = storage_client.bucket(gcs_bucket_name)
blob = bucket.blob(blob_name)
blob.download_to_filename(local_file_path)

# Ler o arquivo DBC localmente usando dbc_reader
rows = [row for row in DbcReader(local_file_path)]

# Agora você pode processar as linhas conforme necessário
print(rows[0:10])



[{'UF_ZI': '290000', 'ANO_CMPT': '2023', 'MES_CMPT': '01', 'ESPEC': '01', 'CGC_HOSP': '', 'N_AIH': '2923101182970', 'IDENT': '1', 'CEP': '46430000', 'MUNIC_RES': '291170', 'NASC': '19810201', 'SEXO': '1', 'UTI_MES_IN': 0, 'UTI_MES_AN': 0, 'UTI_MES_AL': 0, 'UTI_MES_TO': 0, 'MARCA_UTI': '00', 'UTI_INT_IN': 0, 'UTI_INT_AN': 0, 'UTI_INT_AL': 0, 'UTI_INT_TO': 0, 'DIAR_ACOM': 0, 'QT_DIARIAS': 2, 'PROC_SOLIC': '0407030034', 'PROC_REA': '0407030034', 'VAL_SH': 751.13, 'VAL_SP': 245.99, 'VAL_SADT': 0.0, 'VAL_RN': 0.0, 'VAL_ACOMP': 0.0, 'VAL_ORTP': 0.0, 'VAL_SANGUE': 0.0, 'VAL_SADTSR': 0.0, 'VAL_TRANSP': 0.0, 'VAL_OBSANG': 0.0, 'VAL_PED1AC': 0.0, 'VAL_TOT': 997.12, 'VAL_UTI': 0.0, 'US_TOT': 191.75, 'DT_INTER': '20230117', 'DT_SAIDA': '20230119', 'DIAG_PRINC': 'K800', 'DIAG_SECUN': '0000', 'COBRANCA': '12', 'NATUREZA': '00', 'NAT_JUR': '1244', 'GESTAO': '2', 'RUBRICA': 0, 'IND_VDRL': '0', 'MUNIC_MOV': '290520', 'COD_IDADE': '4', 'IDADE': 41, 'DIAS_PERM': 2, 'MORTE': 0, 'NACIONAL': '010', 'NUM_PRO

In [66]:
import pyspark.sql.functions as F

# Converter as linhas para um DataFrame Spark
df = spark.createDataFrame(rows)

# Exibir o DataFrame Spark
df.show()

+--------+--------------------+-------+------+--------+--------------+--------+---------+---------+-----+-------+--------------+--------+---------+-------+----------+----------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+----------+----------+---------+---------+--------+--------+-----+-----+-------+------+------+----------+---------------+---------+---------+---------+--------+-----+-----+--------+--------+------------+------+---------+---------+--------+-----+---------+---------+--------+--------+--------+-------+----------+--------+-------------+----------+----------+----------+--------+-----+--------------------+-------+---------+--------+----+----------------+---------+--------+--------+--------+--------+--------+--------+--------+--------+--------+------+-------+----------+----------+----------+----------+----------+----------+----------+----------+---------+----------+--------+----------+------+--------+----------+----------+-------+-

In [77]:
from google.colab import auth
auth.authenticate_user()


# Caminho para salvar o arquivo CSV no Google Cloud Storage
csv_output_path = "gs://tabsus/tratado/RDBA2301.csv"

# Salvar o DataFrame Spark como um arquivo CSV no Google Cloud Storage
df.write.format("csv").option("header", "true").save(csv_output_path)

print("Arquivo CSV salvo com sucesso em:", csv_output_path)

Py4JJavaError: An error occurred while calling o108.save.
: org.apache.hadoop.fs.UnsupportedFileSystemException: No FileSystem for scheme "gs"
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3443)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource.planForWritingFileFormat(DataSource.scala:454)
	at org.apache.spark.sql.execution.datasources.DataSource.planForWriting(DataSource.scala:530)
	at org.apache.spark.sql.DataFrameWriter.saveToV1Source(DataFrameWriter.scala:388)
	at org.apache.spark.sql.DataFrameWriter.saveInternal(DataFrameWriter.scala:361)
	at org.a

In [100]:
from google.colab import auth
auth.authenticate_user()

# Coletar o DataFrame Spark para a máquina local
df_local = df.toPandas()

# Caminho para salvar o arquivo CSV localmente
local_csv_path = "/content/RDBA2301.csv"

# Salvar o DataFrame como um arquivo CSV localmente
df_local.to_csv(local_csv_path, index=False)

print("Arquivo CSV salvo localmente em:", local_csv_path)

# Transferir o arquivo CSV para o Google Cloud Storage
from google.cloud import storage

# Inicializar o cliente do Google Cloud Storage
storage_client = storage.Client()

# Especificar o caminho no Google Cloud Storage onde o arquivo CSV será salvo
gcs_bucket_name = "tabsus"
gcs_blob_name = "tratado/RDBA2301.csv"

# Fazer upload do arquivo local para o Google Cloud Storage
bucket = storage_client.bucket(gcs_bucket_name)
blob = bucket.blob(gcs_blob_name)
blob.upload_from_filename(local_csv_path)

print("Arquivo CSV transferido com sucesso para:", f"gs://{gcs_bucket_name}/{gcs_blob_name}")


Arquivo CSV salvo localmente em: /content/RDBA2301.csv
Arquivo CSV transferido com sucesso para: gs://tabsus/tratado/RDBA2301.csv
